In [1]:
from pprint import pprint
import sys
sys.path.insert(0, '../')
from utils import check_not_russian_ip

check_not_russian_ip(True)

'Current ip location: NL'

In [1]:
from dotenv import load_dotenv
import os
from pprint import pprint


load_dotenv('../.env')

True

In [2]:
from utils import get_openai_llm, get_openai_embedding_model


openai_llm = get_openai_llm(run_test_question=True)
embeddings = get_openai_embedding_model()

Первый раз человек полетел в космос 12 апреля 1961 года. Это сделал Юрий Гагарин на корабле "Восток-1".


/Users/avalexey/code/pets/book_assistant/notebook/../utils.py:71: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  return OpenAIEmbeddings(model=model)


## Загружаем, сплитуем, готовим RAG

In [3]:
import pandas as pd
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader('data/Machine_Learning_System_Design_v11.pdf')
book = loader.load()

In [4]:
print('pages count:', len(book))
print('max chars on page:', max([len(page.page_content) for page in book]))

pages count: 360
max chars on page: 3723


In [5]:
for page in book:
    page.page_content = page.page_content.replace('Licensed to Alexey Averyanov <averyanovalexey@gmail.com>', '')

In [6]:
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)

splitter = CharacterTextSplitter('\n', chunk_size=1000, chunk_overlap=100)

splited_doc = []
for page in book:
    splited_doc += splitter.create_documents([page.page_content], metadatas=[page.metadata])

len(splited_doc)

997

In [7]:
print(len(splited_doc[2].page_content))
splited_doc[2].page_content

984


'vision, while literature on ML system design is scarce. It leads to the problem we’ve been\nwitnessing in multiple companies, where solid engineers successfully build scattered\nsubcomponents that unfortunately can’t be combined into fully functioning reliable\nsystems.\n\xa0 \xa0 \xa0 \xa0 We decided to contribute to this field and set ourselves a clear goal to share our\nknowledge and experience that we hope will help our readers convert their knowledge\ninto a holistic system.\n\xa0 \xa0 \xa0 \xa0 This book will not be dedicated to a particular technology but will rather represent a\nhigh-level framework on how to approach problems related to building, maintaining, and\nimproving big machine learning systems in various domains, regardless of the size of a\ncompany you work for. The book’s structure will resemble that of a checklist or manual,\nwith ingestion of stories from our own experience. It can be read at once or used at any\nmoment while working on a specific aspect of a mac

In [8]:
from langchain.vectorstores import FAISS


db = FAISS.from_documents(
    splited_doc, embeddings
)  
db.save_local("data/faiss_db") 

In [9]:
db = FAISS.load_local('data/faiss_db', embeddings=embeddings, allow_dangerous_deserialization=True)

rel_docs = db.similarity_search_with_score('Problem space vs solution space', k=5)
rel_docs

[(Document(metadata={'source': 'data/Machine_Learning_System_Design_v11.pdf', 'page': 20}, page_content='What are the problem space and solution space? These are two exploration paradigms\nthat cover different perspectives of a problem. While both are crucial, the former should\nalways precede the latter.\n\xa0 \xa0 \xa0 \xa0 The problem space is often defined with What and Why questions, often even with\nchains of such questions. There is even a popular technique named “Five Whys” that\nrecommends stacking your Why questions on top of each other to dig to the very origin\nof the problem you analyze. Typical questions often look like this:\n\xa0 \xa0 \xa0 \xa0 1. Why do we need to build the solution?    \n2. What problem does it solve?    \n3. Why does the problem occur?    \n4. What are the alternatives we know?    \n5. Why do we want to make it work with given limitations (metrics,\nlatency, number of training samples)?    \n17\n© Manning Publications Co. To comment go to liveBook'),

In [10]:
print(rel_docs[1][0].page_content)

But before you start addressing these questions, there is a paramount action that will
lay the foundation for successful ML system design, and it’s finding and articulating the
problem your solution will solve (or help solve). A seemingly trivial point, especially for
experienced engineers, but based on our own practice in the area, skipping this step in
your preliminary work is deceptively dangerous, and this is what we will try to cover in
this chapter.
          2.1 Problem space vs solution space
        “I suppose it is tempting, if the only tool you have is a hammer, to treat everything
as if it were a nail.”
        Abraham Maslow, American psychologist
        Imagine a boss coming to an engineer with an exciting new idea of a mind-blowing
feature (we’ve all been there). For the matter of visibility, let’s make the example more
specific. Steve works as a machine learning engineer in a growing SaaS company.


In [11]:
retriever = db.as_retriever(
    search_type="similarity", 
    # search_kwargs={
    #     "k": 4,
    #     "score_threshold": 0.6
    # }
)

In [12]:
results = retriever.get_relevant_documents(
    "пространство проблем пространство решений"
)

results

/var/folders/gr/ggbx0psj5sn9zknvwkj1m9n1z1xxgm/T/ipykernel_62674/3208008454.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(


[Document(metadata={'source': 'data/Machine_Learning_System_Design_v11.pdf', 'page': 19}, page_content='But before you start addressing these questions, there is a paramount action that will\nlay the foundation for successful ML system design, and it’s finding and articulating the\nproblem your solution will solve (or help solve). A seemingly trivial point, especially for\nexperienced engineers, but based on our own practice in the area, skipping this step in\nyour preliminary work is deceptively dangerous, and this is what we will try to cover in\nthis chapter.\n\xa0 \xa0 \xa0 \xa0 \xa0 2.1 Problem space vs solution space\n\xa0 \xa0 \xa0 \xa0 “I suppose it is tempting, if the only tool you have is a hammer, to treat everything\nas if it were a nail.”\n\xa0 \xa0 \xa0 \xa0 Abraham Maslow, American psychologist\n\xa0 \xa0 \xa0 \xa0 Imagine a boss coming to an engineer with an exciting new idea of a mind-blowing\nfeature (we’ve all been there). For the matter of visibility, let’s make the

## Агент

In [21]:
# without memory

from langchain.schema import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores import FAISS

# Создаём простой шаблон
template = """
Answer the question based only on the following context and point out pages. 
If there is not information in context, answer 'I don't know'.


{context}

Question: {question}
"""
# Создаём промпт из шаблона
prompt = ChatPromptTemplate.from_template(template)

db = FAISS.load_local('../data/faiss_db', embeddings=embeddings, allow_dangerous_deserialization=True)
retriever = db.as_retriever(
    search_type="similarity",
)


# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    context = "\n\n".join([d.page_content for d in docs])
    pages = ','.join(sorted(str(d.metadata['page']) for d in docs))
    return f"{context}\n\nPages: {pages}"


# Создаём цепочку
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | openai_llm
    | StrOutputParser()
)



In [22]:
pprint(chain.invoke("Что такое пространство проблем расскажи подробнее"))

('Пространство проблем (problem space) определяется вопросами "Что?" и '
 '"Почему?", часто с использованием цепочек таких вопросов. Это этап, на '
 'котором важно понять, какую проблему необходимо решить, прежде чем '
 'переходить к поиску решений. В контексте обсуждаются методы, такие как "Пять '
 'Почему", которые помогают углубиться в корень проблемы. Примеры вопросов, '
 'которые могут задаваться в пространстве проблем, включают: "Почему нам нужно '
 'построить решение?", "Какую проблему оно решает?", "Почему возникает эта '
 'проблема?" и т.д. \n'
 '\n'
 'Ключевой момент заключается в том, что необходимо найти и четко '
 'сформулировать проблему, которую ваше решение должно решить, прежде чем '
 'переходить к этапу решения. Пропуск этого шага может привести к серьезным '
 'проблемам на более поздних стадиях проекта. \n'
 '\n'
 'Таким образом, пространство проблем всегда предшествует пространству '
 'решений, и важно, чтобы инженеры также участвовали в анализе пространства '
 'про